In [1]:
#import necessary packages
import pandas as pd
from fuzzywuzzy import fuzz, process


C:\Users\giuli\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [20]:
PL2020_ratings = pd.read_csv('PL2020_ratings.csv')
PL2020_fl = pd.read_csv('Data/2020-21/Players/FL_20.csv')
PL2020_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2020_fl['player'] = PL2020_fl['name']
PL2020_ratings

,player,position,age,minutes,Team,matchweek,category,Rating,date
0,Joe Bryan,LB,26-361,90.0,Fulham,1.0,Defender,30.040214,2020-09-12
1,Tim Ream,CB,32-343,90.0,Fulham,1.0,Defender,33.442623,2020-09-12
2,Michael Hector,CB,28-055,90.0,Fulham,1.0,Defender,37.665180,2020-09-12
3,Denis Odoi,RB,32-108,90.0,Fulham,1.0,Defender,33.911034,2020-09-12
4,Ainsley Maitland-Niles,WB,23-014,90.0,Arsenal,1.0,Defender,25.875207,2020-09-12
...,...,...,...,...,...,...,...,...,...
10389,Emiliano Martínez,GK,28-263,90.0,Aston Villa,38.0,NaN,63.001016,2021-05-23
10390,Edouard Mendy,GK,29-083,45.0,Chelsea,38.0,NaN,33.585029,2021-05-23
10391,Kepa Arrizabalaga,GK,26-232,45.0,Chelsea,38.0,NaN,26.766151,2021-05-23
10392,Łukasz Fabiański,GK,36-035,90.0,West Ham United,38.0,NaN,71.562860,2021-05-23


In [3]:
PL2020_fl.replace(['Brighton', 'Leeds', 'Leicester', 'Man City', 'Man Utd', 'Newcastle', 'Sheffield Utd','Spurs','West Brom','West Ham', 'Wolves'], ['Brighton & Hove Albion', 'Leeds United', 'Leicester City','Manchester City', 'Manchester United', 'Newcastle United', 'Sheffield United','Tottenham Hotspur', 'West Bromwich Albion', 'West Ham United','Wolverhampton Wanderers'], inplace=True)

In [4]:
players_rating = PL2020_ratings[['player', 'Team']]
players_rating = players_rating.drop_duplicates()
players_rating

,player,Team
0,Joe Bryan,Fulham
1,Tim Ream,Fulham
2,Michael Hector,Fulham
3,Denis Odoi,Fulham
4,Ainsley Maitland-Niles,Arsenal
...,...,...
10190,João Virgínia,Everton
10203,John Ruddy,Wolverhampton Wanderers
10338,Scott Carson,Manchester City
10345,Jack Butland,Crystal Palace


In [5]:
players_fl = PL2020_fl[['player', 'name', 'team']]
players_fl = players_fl.drop_duplicates()
players_fl

,player,name,team
0,Aaron Connolly,Aaron Connolly,Brighton & Hove Albion
1,Aaron Cresswell,Aaron Cresswell,West Ham United
2,Aaron Mooy,Aaron Mooy,Brighton & Hove Albion
3,Aaron Ramsdale,Aaron Ramsdale,Sheffield United
4,Abdoulaye Doucouré,Abdoulaye Doucouré,Everton
...,...,...,...
23955,William Fish,William Fish,Manchester United
24064,Sylvester Jasper,Sylvester Jasper,Fulham
24183,William Osula,William Osula,Sheffield United
24237,Tim Iroegbunam,Tim Iroegbunam,West Bromwich Albion


In [6]:
players_total = pd.merge(players_rating,players_fl , on='player', how='left')
#players_total['test'].isna().sum()
players_total = players_total.sort_values(by='name')

players_total

,player,Team,name,team
255,Aaron Connolly,Brighton & Hove Albion,Aaron Connolly,Brighton & Hove Albion
28,Aaron Cresswell,West Ham United,Aaron Cresswell,West Ham United
519,Aaron Ramsdale,Sheffield United,Aaron Ramsdale,Sheffield United
107,Aaron Wan-Bissaka,Manchester United,Aaron Wan-Bissaka,Manchester United
304,Abdoulaye Doucouré,Everton,Abdoulaye Doucouré,Everton
...,...,...,...,...
529,Ederson,Manchester City,NaN,NaN
532,Adrián,Liverpool,NaN,NaN
537,Caoimhín Kelleher,Liverpool,NaN,NaN
542,Kiko Casilla,Leeds United,NaN,NaN


In [7]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1
players_merged = fuzzy_merge(players_rating, players_fl, 'player', 'player', threshold=90, limit=1)
players_merged

,player,Team,matches
0,Joe Bryan,Fulham,Joe Bryan
1,Tim Ream,Fulham,Tim Ream
2,Michael Hector,Fulham,Michael Hector
3,Denis Odoi,Fulham,Denis Odoi
4,Ainsley Maitland-Niles,Arsenal,Ainsley Maitland-Niles
...,...,...,...
10190,João Virgínia,Everton,João Virgínia
10203,John Ruddy,Wolverhampton Wanderers,John Ruddy
10338,Scott Carson,Manchester City,Scott Carson
10345,Jack Butland,Crystal Palace,Jack Butland


In [105]:
#players_merged = players_merged[['player','Team', 'matches']]
players_merged= players_merged.sort_values(by='matches')
players_merged.reset_index(inplace=True, drop=True)
players_merged.rename(columns={"matches": "name"}, inplace=True)
players_merged



,player,Team,name
0,André Gomes,Everton,
1,Jorginho,Chelsea,
2,Semi Ajayi,West Bromwich Albion,
3,Vitinha,Wolverhampton Wanderers,
4,João Moutinho,Wolverhampton Wanderers,
...,...,...,...
527,Yerry Mina,Everton,Yerry Mina
528,Youri Tielemans,Leicester City,Youri Tielemans
529,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma
530,Zack Steffen,Manchester City,Zack Steffen


In [60]:
players_fl

,player,name,team
0,Aaron Connolly,Aaron Connolly,Brighton
1,Aaron Cresswell,Aaron Cresswell,West Ham
2,Aaron Mooy,Aaron Mooy,Brighton
3,Aaron Ramsdale,Aaron Ramsdale,Sheffield Utd
4,Abdoulaye Doucouré,Abdoulaye Doucouré,Everton
...,...,...,...
23955,William Fish,William Fish,Man Utd
24064,Sylvester Jasper,Sylvester Jasper,Fulham
24183,William Osula,William Osula,Sheffield Utd
24237,Tim Iroegbunam,Tim Iroegbunam,West Brom


In [108]:
out = pd.merge(players_merged,players_fl , on='name', how='left')

out = out[['player_x', 'Team', 'name', 'team']]
out.rename(columns={"player_x": "player_rating", "Team": "team_rating", "name": "player_fl", "team": "team_fl" }, inplace=True)
out

,player_rating,team_rating,player_fl,team_fl
0,André Gomes,Everton,,NaN
1,Jorginho,Chelsea,,NaN
2,Semi Ajayi,West Bromwich Albion,,NaN
3,Vitinha,Wolverhampton Wanderers,,NaN
4,João Moutinho,Wolverhampton Wanderers,,NaN
...,...,...,...,...
547,Yerry Mina,Everton,Yerry Mina,Everton
548,Youri Tielemans,Leicester City,Youri Tielemans,Leicester City
549,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma,Brighton & Hove Albion
550,Zack Steffen,Manchester City,Zack Steffen,Manchester City


In [109]:
#out.to_excel('name_matching_2020.xlsx')

In [27]:
name_matching = pd.read_excel('name_matching_2020.xlsx')
name_matching

,Unnamed: 0,player_rating,team_rating,player_fl,team_fl,Check Player,Check Team
0,0,André Gomes,Everton,André Filipe Tavares Gomes,Everton,False,True
1,1,Jorginho,Chelsea,Jorge Luiz Frello Filho,Chelsea,False,True
2,2,Semi Ajayi,West Bromwich Albion,Oluwasemilogo Adesewo Ibidapo Ajayi,West Bromwich Albion,False,True
3,3,Vitinha,Wolverhampton Wanderers,Vitor Ferreira,Wolverhampton Wanderers,False,True
4,4,João Moutinho,Wolverhampton Wanderers,João Filipe Iria Santos Moutinho,Wolverhampton Wanderers,False,True
...,...,...,...,...,...,...,...
548,548,Youri Tielemans,Leicester City,Youri Tielemans,Leicester City,True,True
549,549,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma,Brighton & Hove Albion,True,True
550,550,Zack Steffen,Manchester City,Zack Steffen,Manchester City,True,True
551,551,Çağlar Söyüncü,Leicester City,Çaglar Söyüncü,Leicester City,False,True


In [28]:
name_matching = name_matching[['player_rating', 'player_fl']]
name_matching = name_matching.drop_duplicates()
name_matching.rename(columns={"player_fl": "player"}, inplace=True)
name_matching

,player_rating,player
0,André Gomes,André Filipe Tavares Gomes
1,Jorginho,Jorge Luiz Frello Filho
2,Semi Ajayi,Oluwasemilogo Adesewo Ibidapo Ajayi
3,Vitinha,Vitor Ferreira
4,João Moutinho,João Filipe Iria Santos Moutinho
...,...,...
548,Youri Tielemans,Youri Tielemans
549,Yves Bissouma,Yves Bissouma
550,Zack Steffen,Zack Steffen
551,Çağlar Söyüncü,Çaglar Söyüncü


In [29]:
PL2020_fl ['date'] = PL2020_fl['kickoff_time'].str.slice(start=0, stop=10)
PL2020_fl

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,player,date
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,1,0,0,0,55,True,0,1,Aaron Connolly,2020-09-14
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,1,0,0,0,50,True,0,1,Aaron Cresswell,2020-09-12
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,0,0,0,0,50,True,0,1,Aaron Mooy,2020-09-14
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,1,0,0,0,50,True,0,1,Aaron Ramsdale,2020-09-14
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,3,0,0,0,55,False,0,1,Abdoulaye Doucouré,2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24360,Aaron Ramsdale,GK,Sheffield Utd,9.0,0,2,30,1,10.0,483,...,9,13618,17381,3763,46,True,0,38,Aaron Ramsdale,2021-05-23
24361,Adrien Silva,MID,Leicester,0.0,0,0,0,0,0.0,220,...,0,-16,0,16,44,True,0,38,Adrien Silva,2021-05-23
24362,Sidnei Tavares,MID,Leicester,0.0,0,0,0,0,0.0,676,...,0,32,60,28,44,True,0,38,Sidnei Tavares,2021-05-23
24363,Kalvin Phillips,MID,Leeds,6.5,0,0,17,0,12.8,204,...,6,210,789,579,49,True,1,38,Kalvin Phillips,2021-05-23


In [30]:
PL2020_fl_matched = pd.merge(PL2020_fl, name_matching , on='player', how='left')
#PL2020_fl_matched['GW']= PL2020_fl_matched['GW'].astype(float)
PL2020_fl_matched['ID'] = PL2020_fl_matched['player_rating'] + '_' + PL2020_fl_matched['date'].astype(str)
PL2020_fl_matched

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,player,date,player_rating,ID
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,0,0,55,True,0,1,Aaron Connolly,2020-09-14,Aaron Connolly,Aaron Connolly_2020-09-14
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,0,0,50,True,0,1,Aaron Cresswell,2020-09-12,Aaron Cresswell,Aaron Cresswell_2020-09-12
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,0,0,50,True,0,1,Aaron Mooy,2020-09-14,NaN,NaN
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,0,0,50,True,0,1,Aaron Ramsdale,2020-09-14,Aaron Ramsdale,Aaron Ramsdale_2020-09-14
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,0,0,55,False,0,1,Abdoulaye Doucouré,2020-09-13,Abdoulaye Doucouré,Abdoulaye Doucouré_2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24394,Adrien Silva,MID,Leicester,0.0,0,0,0,0,0.0,220,...,0,16,44,True,0,38,Adrien Silva,2021-05-23,NaN,NaN
24395,Sidnei Tavares,MID,Leicester,0.0,0,0,0,0,0.0,676,...,60,28,44,True,0,38,Sidnei Tavares,2021-05-23,Sidnei Tavares,Sidnei Tavares_2021-05-23
24396,Kalvin Phillips,MID,Leeds,6.5,0,0,17,0,12.8,204,...,789,579,49,True,1,38,Kalvin Phillips,2021-05-23,Kalvin Phillips,Kalvin Phillips_2021-05-23
24397,Raphael Dias Belloli,MID,Leeds,5.5,1,0,24,0,60.2,570,...,142069,17090,56,True,0,38,Raphael Dias Belloli,2021-05-23,Raphael Dias Belloli,Raphael Dias Belloli_2021-05-23


In [31]:
PL2020_ratings['ID'] = PL2020_ratings['player'] + '_' + PL2020_ratings['date'].astype(str)
PL2020_ratings

,player,position,age,minutes,Team,matchweek,category,Rating,date,ID
0,Joe Bryan,LB,26-361,90.0,Fulham,1.0,Defender,30.040214,2020-09-12,Joe Bryan_2020-09-12
1,Tim Ream,CB,32-343,90.0,Fulham,1.0,Defender,33.442623,2020-09-12,Tim Ream_2020-09-12
2,Michael Hector,CB,28-055,90.0,Fulham,1.0,Defender,37.665180,2020-09-12,Michael Hector_2020-09-12
3,Denis Odoi,RB,32-108,90.0,Fulham,1.0,Defender,33.911034,2020-09-12,Denis Odoi_2020-09-12
4,Ainsley Maitland-Niles,WB,23-014,90.0,Arsenal,1.0,Defender,25.875207,2020-09-12,Ainsley Maitland-Niles_2020-09-12
...,...,...,...,...,...,...,...,...,...,...
10389,Emiliano Martínez,GK,28-263,90.0,Aston Villa,38.0,NaN,63.001016,2021-05-23,Emiliano Martínez_2021-05-23
10390,Edouard Mendy,GK,29-083,45.0,Chelsea,38.0,NaN,33.585029,2021-05-23,Edouard Mendy_2021-05-23
10391,Kepa Arrizabalaga,GK,26-232,45.0,Chelsea,38.0,NaN,26.766151,2021-05-23,Kepa Arrizabalaga_2021-05-23
10392,Łukasz Fabiański,GK,36-035,90.0,West Ham United,38.0,NaN,71.562860,2021-05-23,Łukasz Fabiański_2021-05-23


In [32]:
PL2020_matched = pd.merge(PL2020_ratings, PL2020_fl_matched , on='ID', how='left')
PL2020_matched

,player_x,position_x,age,minutes_x,Team,matchweek_x,category,Rating,date_x,ID,...,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,player_y,date_y,player_rating
0,Joe Bryan,LB,26-361,90.0,Fulham,1.0,Defender,30.040214,2020-09-12,Joe Bryan_2020-09-12,...,0,0,0,50,True,0,1,Joe Bryan,2020-09-12,Joe Bryan
1,Tim Ream,CB,32-343,90.0,Fulham,1.0,Defender,33.442623,2020-09-12,Tim Ream_2020-09-12,...,0,0,0,45,True,0,1,Tim Ream,2020-09-12,Tim Ream
2,Michael Hector,CB,28-055,90.0,Fulham,1.0,Defender,37.665180,2020-09-12,Michael Hector_2020-09-12,...,0,0,0,45,True,1,1,Michael Hector,2020-09-12,Michael Hector
3,Denis Odoi,RB,32-108,90.0,Fulham,1.0,Defender,33.911034,2020-09-12,Denis Odoi_2020-09-12,...,0,0,0,45,True,0,1,Denis Odoi,2020-09-12,Denis Odoi
4,Ainsley Maitland-Niles,WB,23-014,90.0,Arsenal,1.0,Defender,25.875207,2020-09-12,Ainsley Maitland-Niles_2020-09-12,...,0,0,0,50,False,0,1,Ainsley Maitland-Niles,2020-09-12,Ainsley Maitland-Niles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10392,Emiliano Martínez,GK,28-263,90.0,Aston Villa,38.0,NaN,63.001016,2021-05-23,Emiliano Martínez_2021-05-23,...,-62612,15191,77803,53,True,1,38,Emiliano Martínez,2021-05-23,Emiliano Martínez
10393,Edouard Mendy,GK,29-083,45.0,Chelsea,38.0,NaN,33.585029,2021-05-23,Edouard Mendy_2021-05-23,...,41035,58884,17849,53,False,0,38,Edouard Mendy,2021-05-23,Edouard Mendy
10394,Kepa Arrizabalaga,GK,26-232,45.0,Chelsea,38.0,NaN,26.766151,2021-05-23,Kepa Arrizabalaga_2021-05-23,...,416,951,535,47,False,1,38,Kepa Arrizabalaga,2021-05-23,Kepa Arrizabalaga
10395,Łukasz Fabiański,GK,36-035,90.0,West Ham United,38.0,NaN,71.562860,2021-05-23,Łukasz Fabiański_2021-05-23,...,-25881,4342,30223,50,True,0,38,Lukasz Fabianski,2021-05-23,Łukasz Fabiański


In [33]:
PL2020_check = PL2020_matched[['player_x','name' ,'date_x' ,'Team', 'matchweek_x', 'category','position_x', 'position_y', 'minutes_x', 'minutes_y', 'Rating', 'xP', 'bonus', 'bps',  'ict_index', 'influence', 'creativity','threat','total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home' ]]
PL2020_check

,player_x,name,date_x,Team,matchweek_x,category,position_x,position_y,minutes_x,minutes_y,...,ict_index,influence,creativity,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home
0,Joe Bryan,Joe Bryan,2020-09-12,Fulham,1.0,Defender,LB,DEF,90.0,90,...,1.5,10.6,4.3,0.0,1,0,0,0,50,True
1,Tim Ream,Tim Ream,2020-09-12,Fulham,1.0,Defender,CB,DEF,90.0,90,...,1.7,16.4,0.8,0.0,1,0,0,0,45,True
2,Michael Hector,Michael Hector,2020-09-12,Fulham,1.0,Defender,CB,DEF,90.0,90,...,1.7,16.8,0.6,0.0,0,0,0,0,45,True
3,Denis Odoi,Denis Odoi,2020-09-12,Fulham,1.0,Defender,RB,DEF,90.0,90,...,2.1,11.4,2.6,7.0,1,0,0,0,45,True
4,Ainsley Maitland-Niles,Ainsley Maitland-Niles,2020-09-12,Arsenal,1.0,Defender,WB,DEF,90.0,90,...,1.7,10.2,2.8,4.0,6,0,0,0,50,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10392,Emiliano Martínez,Emiliano Martínez,2021-05-23,Aston Villa,38.0,NaN,GK,GK,90.0,90,...,4.2,41.6,0.0,0.0,3,-62612,15191,77803,53,True
10393,Edouard Mendy,Edouard Mendy,2021-05-23,Chelsea,38.0,NaN,GK,GK,45.0,45,...,0.9,8.6,0.0,0.0,1,41035,58884,17849,53,False
10394,Kepa Arrizabalaga,Kepa Arrizabalaga,2021-05-23,Chelsea,38.0,NaN,GK,GK,45.0,45,...,0.0,0.2,0.0,0.0,0,416,951,535,47,False
10395,Łukasz Fabiański,Lukasz Fabianski,2021-05-23,West Ham United,38.0,NaN,GK,GK,90.0,90,...,4.4,44.4,0.0,0.0,9,-25881,4342,30223,50,True


In [34]:
PL2020_check.to_excel('check_2020.xlsx')

In [36]:
PL2020_total = PL2020_matched[['player_x','date_x' ,'Team', 'matchweek_x', 'category','position_x',  'minutes_x', 'Rating', 'xP', 'bonus', 'bps',  'ict_index', 'influence', 'creativity','threat','total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home' ]]
PL2020_total.rename(columns={"player_x": "player", "date_x": "date", "Team": "team", "matchweek_x": "matchweek", "position_x": "position", "minutes_x": "minutes", "Rating": "rating"}, inplace=True)
PL2020_total


C:\Users\giuli\AppData\Local\Temp\ipykernel_28604\2653065548.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2020_total.rename(columns={"player_x": "player", "date_x": "date", "Team": "team", "matchweek_x": "matchweek", "position_x": "position", "minutes_x": "minutes", "Rating": "rating"}, inplace=True)


,player,date,team,matchweek,category,position,minutes,rating,xP,bonus,...,ict_index,influence,creativity,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home
0,Joe Bryan,2020-09-12,Fulham,1.0,Defender,LB,90.0,30.040214,1.1,0,...,1.5,10.6,4.3,0.0,1,0,0,0,50,True
1,Tim Ream,2020-09-12,Fulham,1.0,Defender,CB,90.0,33.442623,0.6,0,...,1.7,16.4,0.8,0.0,1,0,0,0,45,True
2,Michael Hector,2020-09-12,Fulham,1.0,Defender,CB,90.0,37.665180,0.6,0,...,1.7,16.8,0.6,0.0,0,0,0,0,45,True
3,Denis Odoi,2020-09-12,Fulham,1.0,Defender,RB,90.0,33.911034,0.6,0,...,2.1,11.4,2.6,7.0,1,0,0,0,45,True
4,Ainsley Maitland-Niles,2020-09-12,Arsenal,1.0,Defender,WB,90.0,25.875207,3.1,0,...,1.7,10.2,2.8,4.0,6,0,0,0,50,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10392,Emiliano Martínez,2021-05-23,Aston Villa,38.0,NaN,GK,90.0,63.001016,2.5,0,...,4.2,41.6,0.0,0.0,3,-62612,15191,77803,53,True
10393,Edouard Mendy,2021-05-23,Chelsea,38.0,NaN,GK,45.0,33.585029,1.5,0,...,0.9,8.6,0.0,0.0,1,41035,58884,17849,53,False
10394,Kepa Arrizabalaga,2021-05-23,Chelsea,38.0,NaN,GK,45.0,26.766151,0.5,0,...,0.0,0.2,0.0,0.0,0,416,951,535,47,False
10395,Łukasz Fabiański,2021-05-23,West Ham United,38.0,NaN,GK,90.0,71.562860,7.1,2,...,4.4,44.4,0.0,0.0,9,-25881,4342,30223,50,True


In [37]:
PL2020_total.to_csv('PL2020_total.csv')